# Read Dataset and Get Characteristics

In [1]:
import networkx as nx
import numpy as np
import operator

In [2]:
#First, read it in as a normal dataframe
import pandas as pd
df = pd.read_csv(r".\eu_airlines\edges.csv")
df.head()

,# source,target,weight,layer
0,0,1,1,1
1,0,37,1,1
2,0,251,1,3
3,0,70,1,4
4,0,63,1,14


In [3]:
df = df.iloc[:, 0:2]
df.rename(columns={'# source':'s', ' target':'t'}, inplace=True)
df

,s,t
0,0,1
1,0,37
2,0,251
3,0,70
4,0,63
...,...,...
3583,383,396
3584,385,413
3585,409,420
3586,435,444


In [4]:
#Convert it to a graph
#Each row is an edge with a source and a target. The network will automatically create nodes for you based on their names.
g = nx.from_pandas_edgelist(df, source='s', target='t')
print(nx.info(g))

Graph with 417 nodes and 2953 edges


In [5]:
g = g.to_undirected()

In [6]:
print(nx.info(g))

Graph with 417 nodes and 2953 edges


# Get Simple Graph
Remove self-loops

In [7]:
g.remove_edges_from(nx.selfloop_edges(g))

In [8]:
print(nx.info(g))

Graph with 417 nodes and 2953 edges


# Check connectedness

In [9]:
nx.is_connected(g)

True

In [10]:
nx.number_connected_components(g)

1

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [11]:
from myTruss import mappingAndRelabeling

In [12]:
g_relabled = mappingAndRelabeling(g)

In [13]:
"""Before""" 
print(nx.info(g))

Graph with 417 nodes and 2953 edges


In [14]:
"""After""" 
print(nx.info(g_relabled))

Graph with 417 nodes and 2953 edges


In [15]:
g = g_relabled

# Getting Community Structure According to InfoMap

In [16]:
import community
import infomap

In [17]:
im = infomap.Infomap("--two-level")

print("Building Infomap network from a NetworkX graph...")
for source, target in g.edges:
    im.add_link(source, target)

print("Find communities with Infomap...")
im.run()

print(f"Found {im.num_top_modules} modules with codelength: {im.codelength}")

communities = im.get_modules()

Building Infomap network from a NetworkX graph...
Find communities with Infomap...
Found 10 modules with codelength: 7.445887560302869


In [19]:
modularity_value = community.modularity(communities, g)
modularity_value

0.10860198419556237

# Rename the communities

In [20]:
from organizeCommunities import *

def flip_nodes_and_communities(dict_nodes_communities):
    new_dict = {}
    for k, v in dict_nodes_communities.items():
        new_dict[v]=[]
    
    for kk,vv in new_dict.items():
        for k,v in dict_nodes_communities.items():
            if dict_nodes_communities[k] == kk:
                new_dict[kk].append(k)
    
    return new_dict

In [21]:
infomap_communities = flip_nodes_and_communities(communities)

In [22]:
infomap_communities_organized = orderCommunities(infomap_communities)

In [24]:
communityInfo(infomap_communities, infomap_communities_organized)

Number of partitions:  10
Number of nodes in the communities detected:  417
Number of repetitions:  0




In [25]:
dict_graph = dict()
for i in g:
    dict_graph[i] = []
    for j in g[i]:
        dict_graph[i].append(j)

In [26]:
print('Raw graph information:')
print(nx.info(g))

Raw graph information:
Graph with 417 nodes and 2953 edges


In [29]:
from graphsExtractor import *

In [30]:
dict_graph = dict()
for i in g:
    dict_graph[i] = []
    for j in g[i]:
        dict_graph[i].append(j)

In [31]:
# Get the intra graph (local component)
g_intra_wo, graph_wo_intra = intra_wo(dict_graph, infomap_communities_organized)
print(nx.info(graph_wo_intra))

Graph with 417 nodes and 2738 edges


In [32]:
# Get the inter graph (global component)
g_inter, graph_inter = inter_wo_o(g, infomap_communities_organized)
print(nx.info(graph_inter))

Graph with 417 nodes and 215 edges


In [33]:
g_intra_wo_converted = nx.Graph(g_intra_wo)
g_inter_converted = nx.Graph(g_inter)
dict_graph_converted = nx.Graph(dict_graph)
listofnodes = list(dict_graph_converted.nodes())

# Community-aware centrality 1: Comm Centrality

Note: Comm Centrality will give at each runtime a value due to the random constant number `R`. One can average its values after 100 iterations.

In [34]:
from CommCentralityCode import *

In [35]:
dict_com = comm_centreality(g, communities)

In [36]:
dict_com

{17: 1448.0279768627056,
 307: 1393.1593406593408,
 22: 1261.7878240322348,
 240: 1202.825450180072,
 252: 1169.5555555555554,
 91: 1155.7147511876076,
 26: 1133.387752925041,
 225: 1013.3333333333334,
 400: 1013.3333333333334,
 1: 935.0882492531997,
 23: 925.1304283967304,
 366: 779.0,
 367: 779.0,
 350: 661.8333333333334,
 2: 542.8131901789143,
 63: 534.2779151590834,
 59: 525.0313672209334,
 21: 380.05554451802703,
 92: 323.3899064066573,
 52: 253.77661314102846,
 43: 248.08642979447455,
 48: 242.39624644792067,
 47: 242.04060998876105,
 101: 239.90679123380335,
 308: 236.64186813186816,
 168: 231.0158797548129,
 372: 210.58333333333334,
 83: 166.11021480287593,
 3: 161.48694083380087,
 5: 154.72984810976814,
 10: 151.17348351817196,
 12: 149.03966476321423,
 96: 146.5502095490969,
 4: 145.12766371245846,
 24: 143.34948141666035,
 140: 137.30366161094685,
 37: 136.236752233468,
 25: 134.81420639682952,
 187: 133.74729701935067,
 159: 131.96911472355256,
 90: 130.69074826494543,
 207

In [37]:
dict_com[1]

935.0882492531997

# Community-aware centrality 2: CBM - Community-based Mediator

In [38]:
from CBMCode import *

In [39]:
dict_cbm = CBM(g, communities)

In [40]:
dict_cbm[1]

0.0012931880450018501

# Community-aware centrality 3: Community Hub-Bridge centrality

In [44]:
from CHBCode import *

In [45]:
dict_chb = community_Hub_Bridge(g,communities)

In [47]:
dict_chb[0]

0.7961630695443646

# Community-aware centrality 4: K-shell with community centrality

In [48]:
from KshellWithCommunityCode import *

In [49]:
dict_kshell = kshell_community(g,g_intra_wo,g_inter,alpha=0.5)
dict_kshell[1]

13.5

# Community-aware centrality 5: Community Based Centrality 

In [50]:
from CommunityBasedCentralityCode import *

In [51]:
dict_cbc = commmunityBasedCentrality(g, communities)

In [53]:
dict_cbc[1]

66.58273381294957

# Community-aware centrality 6: Participation Coefficient

In [54]:
import ParticipationCoefficientCode

In [55]:
dict_pc = ParticipationCoefficientCode.getPCFinal(g, communities)

In [57]:
dict_pc[1]

0.1649559231976815